<a href="https://colab.research.google.com/github/yurystepanets/ds_find_number/blob/main/Copy_of_ML7_Hyperparams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**ML-7. Оптимизация гиперпараметров модели**

In [ ]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

%matplotlib inline
plt.style.use('seaborn')

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
data = pd.read_csv('./drive/MyDrive/_train_sem09 (1).csv')

#ЕСЛИ РАБОТАЕМ В GOOGLE COLAB
# подключаем google диск, где у нас хранятся данные
# from google.colab import drive
# drive.mount('/content/drive')
# # берем необходимый нам файл с диска, указав путь
# data = pd.read_csv('./drive/MyDrive/SF/SF module 7/adult.csv')
data.head()


,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X=data.drop(['Activity'],axis=1)
y=data['Activity']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 1, test_size = 0.2)

#### **1. Логистическая регрессия**

In [ ]:
#Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(max_iter = 50)
#Обучаем модель, минимизируя logloss
log_reg.fit(X_train, y_train)
print("accuracy на тестовом наборе: {:.2f}".format(log_reg.score(X_test, y_test)))
y_test_pred = log_reg.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

accuracy на тестовом наборе: 0.76
f1_score на тестовом наборе: 0.79


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### **1.2 Логистическая регрессия - Оптимизация GridSearchCV**

In [ ]:
param_grid = {'penalty': ['l2', 'none'] ,
              'solver': ['lbfgs', 'sag'],
               'C': list(np.linspace(0.01, 1, 10, dtype=float))},
            
grid_search_2 = GridSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=50), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search_2.fit(X_train, y_train) 
#print("accuracy на тестовом наборе: {:.2f}".format(grid_search_2.score(X_test_scaled, y_test)))
y_test_pred = grid_search_2.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_2.best_params_))

CPU times: user 3.47 s, sys: 218 ms, total: 3.69 s
Wall time: 2min 52s
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'C': 0.12, 'penalty': 'l2', 'solver': 'sag'}


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


#### **1.3 Логистическая регрессия - Оптимизация RandomizedSearchCV**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

param_distributions = {'penalty': ['l2', 'none'] ,
              'solver': ['lbfgs', 'sag'],
               'C': list(np.linspace(0.01, 1, 10, dtype=float))},
            
random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=50), 
    param_distributions=param_distributions, 
    cv=5, 
    n_iter = 10, 
    n_jobs = -1
)  
%time random_search.fit(X_train, y_train) 
y_test_pred = random_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

CPU times: user 870 ms, sys: 189 ms, total: 1.06 s
Wall time: 36.2 s
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.12}


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### **1.4 Логистическая регрессия - Оптимизация Hyperopt**

In [ ]:
#делаем импорт и выведем версию библиотеки
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
# fmin - основная функция, она будет минимизировать наш функционал
# tpe - алгоритм оптимизации
# hp - включает набор методов для объявления пространства поиска гиперпараметров
# trails - используется для логирования результатов

print("Версия Hyperopt : {}".format(hyperopt.__version__))


Версия Hyperopt : 0.2


In [ ]:
#Напишем функции для убыстрения расчета модельных данных
def hyperopt_lr(params, cv=5, X=X_train, y=y_train, random_state=42): #функция модели линейная регрессия
    # используем эту комбинацию для построения модели
    model_lr = linear_model.LogisticRegression(**params, random_state=random_state)
    # обучаем модель
    model_lr.fit(X, y)
    score = metrics.f1_score(y,model_lr.predict(X))
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model_lr, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()
    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

# начинаем подбор гиперпараметров для модели логистической регрессии
space_lr = hp.choice(
    'logreg_params',
    [
        {
            'penalty': hp.choice('penalty_1', ['l2', 'none']),
            'solver': hp.choice('solver_1', ['lbfgs',  'sag']), #'newton-cg',
            'C': hp.choice('С_1', list(np.linspace(0.01, 1, 10, dtype=float)))
        }
        # ,
        # {
        #     'penalty': hp.choice('penalty_2', ['l1', 'l2']),
        #     'solver': hp.choice('solver_2', ['liblinear', 'saga']),
        #     'C': hp.choice('С_2', list(np.linspace(0.01, 1, 10, dtype=float)))
        # }
    ]
)

random_state = 42
trials = Trials() # используется для логирования результатов
best=fmin(hyperopt_lr, # наша функция
          space=space_lr, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=50, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.RandomState(random_state)
          #rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

  0%|          | 0/50 [00:00<?, ?it/s, best loss: ?]

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



  2%|▏         | 1/50 [00:03<02:29,  3.06s/it, best loss: -0.7677872649369261]

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



  4%|▍         | 2/50 [00:23<10:40, 13.34s/it, best loss: -0.7677872649369261]

...

 94%|█████████▍| 47/50 [10:14<00:10,  3.65s/it, best loss: -0.7831177392651217]

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



 98%|█████████▊| 49/50 [10:19<00:03,  3.04s/it, best loss: -0.7831177392651217]

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



100%|██████████| 50/50 [10:22<00:00, 12.46s/it, best loss: -0.7831177392651217]
Наилучшие значения гиперпараметров {'logreg_params': 0, 'penalty_1': 0, 'solver_1': 0, 'С_1': 1}


#### **1.5 Логистическая регрессия - Оптимизация Optuna**

In [ ]:
%pip install optuna

In [ ]:
import optuna
print("Версия Optuna: {}".format(optuna.__version__))

Версия Optuna: 3.0.5


In [ ]:
random_state=42
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  params1 = {
    'penalty': trial.suggest_categorical(name='penalty', choices=['l2', 'none']), 
    'solver': trial.suggest_categorical(name='solver', choices=['lbfgs',  'sag']), 
    'C': trial.suggest_float('C1', 0.01, 1, step=0.1) 
  }


  # создаем модель
  model = linear_model.LogisticRegression(**params1, random_state=random_state)

  # обучаем модель
  model.fit(X_train, y_train)
  score = metrics.f1_score(y_train, model.predict(X_train))

  return score

# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=50)

[I 2022-12-25 08:14:40,022] A new study created in memory with name: LogisticRegression
/usr/local/lib/python3.8/dist-packages/optuna/distributions.py:668: UserWarning: The distribution is specified by [0.01, 1] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.01, 0.91].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2022-12-25 08:14:44,440] Trial 0 finished with value: 0.8856269113149847 and parameters: {'penalty': 'none', 'solver': 'sag', 'C1': 0.21000000000000002}. Best is trial 0 with value: 0.8856269113149847.
/usr/local/lib/python3.8/dist-packages/optuna/distributions.py:668: UserWarning: The distribution is specified by [0.01, 1] a

In [ ]:
# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study.best_value))

Наилучшие значения гиперпараметров {'penalty': 'none', 'solver': 'lbfgs', 'C1': 0.81}
f1_score на обучающем наборе: 0.91


#### **2. Cлучайный лес**

In [ ]:
#Создаём объект класса RandomForestClassifier
rf = ensemble.RandomForestClassifier(
    n_estimators=100, #число деревьев
    criterion='entropy', #критерий эффективности
    max_depth=12, #максимальная глубина дерева
    max_features='sqrt', #число признаков из метода случайных подпространств
    random_state=42 #генератор случайных чисел
)
#Обучаем модель 
rf.fit(X_train, y_train)
#Делаем предсказание класса
y_test_pred = rf.predict(X_test)
#Выводим отчёт о метриках
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.82


#### **2.1 Cлучайный лес - Оптимизация GridSearchCV**

In [ ]:
param_distributions = {'n_estimators': list(range(80, 200, 30)),
               'min_samples_leaf':  [5, 10, 15],
               'max_depth': list(np.linspace(20, 40, 10, dtype=int))
               }
            
grid_search_2 = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_grid=param_distributions, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search_2.fit(X_train, y_train) 
y_test_pred = grid_search_2.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_2.best_params_))

CPU times: user 6.42 s, sys: 513 ms, total: 6.93 s
Wall time: 9min 3s
f1_score на тестовом наборе: 0.82
Наилучшие значения гиперпараметров: {'max_depth': 20, 'min_samples_leaf': 5, 'n_estimators': 140}


#### **2.2 Cлучайный лес - Оптимизация RandomizedSearchCV**

In [ ]:
param_distributions = {'n_estimators': list(range(80, 200, 30)),
              'min_samples_leaf': [5, 10, 15],
              'max_depth': list(np.linspace(1, 40, 10, dtype=int))
              }
            
random_search_forest = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_distributions, 
    cv=5,
    n_iter = 50, 
    n_jobs = -1
)  
%time random_search_forest.fit(X_train, y_train) 
y_train_pred = random_search_forest.predict(X_train)
#print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
#print("accuracy на тестовом наборе: {:.2f}".format(random_search_forest.score(X_test, y_test)))
y_test_pred = random_search_forest.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_forest.best_params_))

CPU times: user 3.43 s, sys: 204 ms, total: 3.63 s
Wall time: 3min 20s
f1_score на тестовом наборе: 0.82
Наилучшие значения гиперпараметров: {'n_estimators': 110, 'min_samples_leaf': 5, 'max_depth': 14}


#### **2.3 Cлучайный лес - Оптимизация Hyperopt**

In [ ]:
#делаем импорт и выведем версию библиотеки
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
# fmin - основная функция, она будет минимизировать наш функционал
# tpe - алгоритм оптимизации
# hp - включает набор методов для объявления пространства поиска гиперпараметров
# trails - используется для логирования результатов

print("Версия Hyperopt : {}".format(hyperopt.__version__))


Версия Hyperopt : 0.2


In [ ]:
#делаем импорт и выведем версию библиотеки
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
# fmin - основная функция, она будет минимизировать наш функционал
# tpe - алгоритм оптимизации
# hp - включает набор методов для объявления пространства поиска гиперпараметров
# trails - используется для логирования результатов
# зададим пространство поиска гиперпараметров
space_rf={'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
       'max_depth' : hp.quniform('max_depth', 15, 26, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1)
      }

def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=42): 
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)
    # обучаем модель
    model.fit(X, y)
    score = metrics.f1_score(y, model.predict(X))
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()
    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

random_state = 42
trials = Trials() # используется для логирования результатов
best=fmin(hyperopt_rf, # наша функция
          space=space_rf, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=50, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.RandomState(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|██████████| 50/50 [08:01<00:00,  9.63s/it, best loss: -0.8125761394848935]
Наилучшие значения гиперпараметров {'max_depth': 26.0, 'min_samples_leaf': 2.0, 'n_estimators': 186.0}


#### **2.4 Cлучайный лес - Оптимизация Optuna**

In [ ]:
%pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import optuna
print("Версия Optuna: {}".format(optuna.__version__))

Версия Optuna: 3.0.5


In [ ]:
import optuna
random_state=42
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 50)
  max_depth = trial.suggest_int('max_depth', 10, 30, 5)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 2)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train, y_train)
  score = metrics.f1_score(y_train, model.predict(X_train))

  return score
  

# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=50)

[I 2022-12-25 08:36:35,584] A new study created in memory with name: RandomForestClassifier
[I 2022-12-25 08:36:37,963] Trial 0 finished with value: 0.8856088560885609 and parameters: {'n_estimators': 150, 'max_depth': 10, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.8856088560885609.
[I 2022-12-25 08:36:39,704] Trial 1 finished with value: 0.9511301160659744 and parameters: {'n_estimators': 100, 'max_depth': 15, 'min_samples_leaf': 4}. Best is trial 1 with value: 0.9511301160659744.
[I 2022-12-25 08:36:41,095] Trial 2 finished with value: 0.9008567931456548 and parameters: {'n_estimators': 100, 'max_depth': 20, 'min_samples_leaf': 10}. Best is trial 1 with value: 0.9511301160659744.
[I 2022-12-25 08:36:46,533] Trial 3 finished with value: 0.9587282176704371 and parameters: {'n_estimators': 200, 'max_depth': 20, 'min_samples_leaf': 4}. Best is trial 3 with value: 0.9587282176704371.
[I 2022-12-25 08:36:53,313] Trial 4 finished with value: 0.9926470588235294 and parameters: {'n